# Python project - graphs - Omer Vinik

## Part I – The Node class
## Task 1 – Define the class
### Implement the Node class

In [1]:
class Node:
    def __init__(self, name):
        self.name = name
        self.neighbors = {}

    def __str__(self):
        out = "Node name: " + self.name + "\n"
        if not self.is_isolated():
            out += "Node neighbors:\n"
            for neighbor in self.neighbors:
                s = "Neighbor name: {:<9} Weight: {}\n"
                out += s.format(neighbor, self.neighbors[neighbor])
        else:
            out += "Node has no neighbors.\n"            
        return out

    def __len__(self):
        return len(self.neighbors)

    def __contains__(self, item):
        return item in self.neighbors

    def __getitem__(self, key):
        return self.neighbors[key] if self.is_neighbor(key) else None

    def __eq__(self, other):
        return self.name == other.name

    def __ne__(self, other):
        return not (self == other)

    def is_neighbor(self, name):
        return name in self

    def add_neighbor(self, name, weight=1):
        if self.name != name:
            if self.is_neighbor(name):
                self.neighbors[name] = max(weight, self.get_weight(name))
            else:
                self.neighbors[name] = weight

    def remove_neighbor(self, name):
        if self.is_neighbor(name):
            self.neighbors.pop(name)

    def get_weight(self, name):
        return self[name]

    def is_isolated(self):
        return not len(self)

## Task 2 – Exemplary usage
## Question 1
### Create 10 Node objects according to the figure above, print them (textually, of course).

In [2]:
node_01 = Node("1")
node_01.add_neighbor("2",10)
node_01.add_neighbor("4",20)
node_01.add_neighbor("5",20)
node_01.add_neighbor("6",5)
node_01.add_neighbor("7",15)
print node_01

node_02 = Node("2")
node_02.add_neighbor("3",5)
node_02.add_neighbor("4",10)
print node_02

node_03 = Node("3")
node_03.add_neighbor("2",15)
node_03.add_neighbor("4",5)
print node_03

node_04 = Node("4")
node_04.add_neighbor("5",10)
print node_04

node_05 = Node("5")
node_05.add_neighbor("6",5)
print node_05

node_06 = Node("6")
print node_06

node_07 = Node("7")
node_07.add_neighbor("6",10)
print node_07

node_08 = Node("8")
node_08.add_neighbor("1",5)
node_08.add_neighbor("7",5)
node_08.add_neighbor("2",20)
print node_08

node_09 = Node("9")
node_09.add_neighbor("2",15)
node_09.add_neighbor("8",20)
node_09.add_neighbor("10",10)
print node_09

node_10 = Node("10")
node_10.add_neighbor("2",5)
node_10.add_neighbor("3",15)
print node_10

Node name: 1
Node neighbors:
Neighbor name: 2         Weight: 10
Neighbor name: 5         Weight: 20
Neighbor name: 4         Weight: 20
Neighbor name: 7         Weight: 15
Neighbor name: 6         Weight: 5

Node name: 2
Node neighbors:
Neighbor name: 3         Weight: 5
Neighbor name: 4         Weight: 10

Node name: 3
Node neighbors:
Neighbor name: 2         Weight: 15
Neighbor name: 4         Weight: 5

Node name: 4
Node neighbors:
Neighbor name: 5         Weight: 10

Node name: 5
Node neighbors:
Neighbor name: 6         Weight: 5

Node name: 6
Node has no neighbors.

Node name: 7
Node neighbors:
Neighbor name: 6         Weight: 10

Node name: 8
Node neighbors:
Neighbor name: 1         Weight: 5
Neighbor name: 2         Weight: 20
Neighbor name: 7         Weight: 5

Node name: 9
Node neighbors:
Neighbor name: 8         Weight: 20
Neighbor name: 2         Weight: 15
Neighbor name: 10        Weight: 10

Node name: 10
Node neighbors:
Neighbor name: 3         Weight: 15
Neighbor name: 

## Part II – The Graph class
## Task 1 – Define the class
### Implement the Graph class

In [3]:
class Graph:
    def __init__(self, name, nodes=[]):
        self.name = name
        self.nodes = {}
        for node in nodes:
            self.add_node(node)

    def __str__(self):
        out = "Graph name: {}\n".format(self.name)
        if len(self):
            out += "Graph's nodes:\n"
            for node in self.nodes.values():
                out += "{}\n".format(str(node))
        else:
            out += "Graph has no nodes.\n"
        return out

    def __len__(self):
        return len(self.nodes)

    def __contains__(self, item):
        return item in self.nodes.values() if isinstance(item, Node) else item in self.nodes

    def __getitem__(self, key):
        if key in self:
            return self.nodes[key]
        else:
            raise KeyError("Node {} not in {}".format(key, self.name))

    def __add__(self, other):
        new_graph = Graph("{} + {}".format(self.name, other.name), self.nodes.values())
        for node in other.nodes.values():
            new_graph.add_node(node)
        return new_graph

    def add_node(self, node):
        # If node not in graph
        if node not in self:
            # Add node with all its data
            self.nodes[node.name] = node
        else:
            # (node exists)
            # Run through node's neighbors
            for neighbor, weight in node.neighbors.items():
                # update weight
                self[node.name].add_neighbor(neighbor, weight)

    def remove_node(self, name):
        if name in self:
            self.nodes.pop(name)

    def is_edge(self, frm_name, to_name):
        return self[frm_name].is_neighbor(to_name) if frm_name in self and to_name in self else False

    def add_edge(self, frm_name, to_name, weight=1):
        if frm_name in self:
            self[frm_name].add_neighbor(to_name, weight)

    def remove_edge(self, frm_name, to_name):
        if frm_name in self and self[frm_name].is_neighbor(to_name):
            self[frm_name].remove_neighbor(to_name)

    def get_edge_weight(self, frm_name, to_name):
        if frm_name in self and to_name in self:
            return self[frm_name].get_weight(to_name) if self[frm_name].is_neighbor(to_name) else None

    def get_path_weight(self, path):
        if path:
            weights = [self.get_edge_weight(i, j) for i, j in zip(path[:-1], path[1:])]
            return sum(weights) if all(weights) else None

    def is_reachable(self, frm_name, to_name):
        if frm_name in self and to_name in self:
            visited = set()
            stack = [frm_name]
            
            while stack:
                node = stack.pop()
                if node in visited:
                    continue
                visited.add(node)
                if self[node] == self[to_name]:
                    return True
                for n in self[node].neighbors:
                    if n not in visited and n not in stack:
                        stack.append(n)
            return False

    def find_shortest_path(self, frm_name, to_name):
        if frm_name in self:
            visited = set()
            stack = [frm_name]
            weights = dict(zip(self.nodes, [0] * len(self)))
            paths = dict(zip(self.nodes, [[]] * len(self)))
            paths[frm_name] = [frm_name]
            
            while stack:
                node = stack.pop()
                if node in visited:
                    continue
                visited.add(node)
                for n in self[node].neighbors:
                    if n not in visited:
                        stack.append(n)
                    edge_w = weights[node] + int(self.get_edge_weight(node, n))
                    if weights[n] == 0 or weights[n] > edge_w:
                        weights[n] = edge_w
                        paths[n] = paths[node] + []
                        paths[n].append(n)

            return paths[to_name]

In [4]:
graph = Graph("Graph01", [node_01, node_02, node_03, node_04, node_05, node_06, node_07, node_08, node_09, node_10])
print graph

Graph name: Graph01
Graph's nodes:
Node name: 10
Node neighbors:
Neighbor name: 3         Weight: 15
Neighbor name: 2         Weight: 5

Node name: 1
Node neighbors:
Neighbor name: 2         Weight: 10
Neighbor name: 5         Weight: 20
Neighbor name: 4         Weight: 20
Neighbor name: 7         Weight: 15
Neighbor name: 6         Weight: 5

Node name: 3
Node neighbors:
Neighbor name: 2         Weight: 15
Neighbor name: 4         Weight: 5

Node name: 2
Node neighbors:
Neighbor name: 3         Weight: 5
Neighbor name: 4         Weight: 10

Node name: 5
Node neighbors:
Neighbor name: 6         Weight: 5

Node name: 4
Node neighbors:
Neighbor name: 5         Weight: 10

Node name: 7
Node neighbors:
Neighbor name: 6         Weight: 10

Node name: 6
Node has no neighbors.

Node name: 9
Node neighbors:
Neighbor name: 8         Weight: 20
Neighbor name: 2         Weight: 15
Neighbor name: 10        Weight: 10

Node name: 8
Node neighbors:
Neighbor name: 1         Weight: 5
Neighbor name: 2

## Question 3
### How many edges are in the graph, and what is their total weight?

In [5]:
num_of_edges, total_weight = 0, 0
for node in graph.nodes:
    num_of_edges += len(graph[node])
    for neighbor in graph[node].neighbors:
        total_weight += graph[node].get_weight(neighbor)
        
print "Number of edges:", num_of_edges
print "Their total weight:", total_weight

Number of edges: 20
Their total weight: 225


## Question 4
### Sort the nodes by the number of their neighbors.

In [6]:
res = []
for node in graph.nodes:
    res.append((node, len(graph[node])))
print sorted(res, key=lambda x: x[1])

[('6', 0), ('5', 1), ('4', 1), ('7', 1), ('10', 2), ('3', 2), ('2', 2), ('9', 3), ('8', 3), ('1', 5)]


## Task 2
## Question 1
### Create 3 Graph objects, each contains a different collection of nodes, which together contain all 10 nodes. Use the __add()__ method to create a total graph that contains the entire data of the example.


In [7]:
graph1 = Graph("Graph01", [node_01, node_03, node_05, node_07])
graph2 = Graph("Graph02", [node_02, node_08, node_09])
graph3 = Graph("Graph03", [node_04, node_06, node_10])
graph = graph1 + graph2 + graph3
print graph

Graph name: Graph01 + Graph02 + Graph03
Graph's nodes:
Node name: 10
Node neighbors:
Neighbor name: 3         Weight: 15
Neighbor name: 2         Weight: 5

Node name: 1
Node neighbors:
Neighbor name: 2         Weight: 10
Neighbor name: 5         Weight: 20
Neighbor name: 4         Weight: 20
Neighbor name: 7         Weight: 15
Neighbor name: 6         Weight: 5

Node name: 3
Node neighbors:
Neighbor name: 2         Weight: 15
Neighbor name: 4         Weight: 5

Node name: 2
Node neighbors:
Neighbor name: 3         Weight: 5
Neighbor name: 4         Weight: 10

Node name: 5
Node neighbors:
Neighbor name: 6         Weight: 5

Node name: 4
Node neighbors:
Neighbor name: 5         Weight: 10

Node name: 7
Node neighbors:
Neighbor name: 6         Weight: 10

Node name: 6
Node has no neighbors.

Node name: 9
Node neighbors:
Neighbor name: 8         Weight: 20
Neighbor name: 2         Weight: 15
Neighbor name: 10        Weight: 10

Node name: 8
Node neighbors:
Neighbor name: 1         Weight

## Question 3
### Sort the nodes by the number of their reachable nodes.

In [8]:
res = []
for frm_name in graph.nodes:
    reachable_nodes = 0
    for to_name in graph.nodes:
        if frm_name == to_name:
            continue
        if graph.is_reachable(frm_name, to_name):
            reachable_nodes += 1
    res.append((frm_name, reachable_nodes))
print sorted(res, key=lambda x: x[1])

[('6', 0), ('5', 1), ('7', 1), ('4', 2), ('3', 4), ('2', 4), ('10', 5), ('1', 6), ('8', 7), ('9', 9)]


## Question 4
### What is the pair of nodes that the shortest path between them has the highest weight?

In [9]:
res = []
for frm_name in graph.nodes:
    for to_name in graph.nodes:
        if frm_name == to_name:
            continue
        shortest_path = graph.find_shortest_path(frm_name, to_name)
        res.append((frm_name, to_name, graph.get_path_weight(shortest_path)))
print sorted(res, key=lambda x: x[2])[-1]

('9', '5', 35)


## Task 3 – The roadmap implementation

In [10]:
from datetime import datetime
def get_data(file_name, time_stmp_format):
    res = []
    with open(file_name) as f:
        for line in f:
            fields = line.rstrip().split(',')
            if len(fields) == 4 and fields[0] in ['North', 'South', 'East', 'West', 'Center']:
                from_area, to_area = fields[0], fields[2]
                try:
                    time_start = datetime.strptime(fields[1], time_stmp_format)
                    time_end = datetime.strptime(fields[3], time_stmp_format)
                except ValueError as err:
                    # for double checking input vs. format
                    print err
                    continue
                res.append(((from_area, to_area), (time_end - time_start).seconds))
    print
    return res

def print_data(res_list):
    res_dict = {}
    for road in res:
        key, value = road[0], road[1]
        l = res_dict.get(key, [])
        l.append(value)
        res_dict[key] = l
    for road in res_dict:
        print road, "-", int(round(sum(res_dict[road])/float(len(res_dict[road]))))
        
res = get_data("travelsEW.csv", "%d/%m/%Y %Hh%Mm")
print_data(res)
print
res = get_data("travelsWE.csv", "%I:%M:%S%p ; %b %d %y")
print_data(res)

time data '22/01/2016 25h52m' does not match format '%d/%m/%Y %Hh%Mm'
time data '26/02/2016 1412' does not match format '%d/%m/%Y %Hh%Mm'

('Center', 'West') - 5378
('Center', 'North') - 5308
('East', 'South') - 3597

time data '05:55:00PM ; Jam 25 16' does not match format '%I:%M:%S%p ; %b %d %y'
time data '08:12:00PM : Feb 10 16' does not match format '%I:%M:%S%p ; %b %d %y'
time data '02:43:00AM ; Feb 00 16' does not match format '%I:%M:%S%p ; %b %d %y'

('South', 'East') - 3582
('Center', 'South') - 10806
('North', 'Center') - 3543
('West', 'North') - 3565
('Center', 'East') - 3513
('West', 'Center') - 8953


## Question 1
### From each file create a graph whose nodes are the country regions, and whose edges are the roads themselves.

In [11]:
node_c = Node("Center")
node_c.add_neighbor("West", 5378)
node_c.add_neighbor("North", 5308)
node_e = Node("East")
node_e.add_neighbor("South", 3597)
graph1 = Graph("Graph1", [node_c, node_e])

node_s = Node("South")
node_s.add_neighbor("East", 3582)
node_c = Node("Center")
node_c.add_neighbor("South", 10806)
node_n = Node("North")
node_n.add_neighbor("Center", 3543)
node_w = Node("West")
node_w.add_neighbor("North", 3565)
node_c.add_neighbor("East", 3513)
node_w.add_neighbor("Center", 8953)
graph2 = Graph("Graph2", [node_s, node_c, node_n, node_w])

### When the two graphs are ready, add them together to create the complete graph of the roadmap.

In [12]:
graph = graph1 + graph2
print graph

Graph name: Graph1 + Graph2
Graph's nodes:
Node name: West
Node neighbors:
Neighbor name: North     Weight: 3565
Neighbor name: Center    Weight: 8953

Node name: East
Node neighbors:
Neighbor name: South     Weight: 3597

Node name: North
Node neighbors:
Neighbor name: Center    Weight: 3543

Node name: Center
Node neighbors:
Neighbor name: West      Weight: 5378
Neighbor name: East      Weight: 3513
Neighbor name: North     Weight: 5308
Neighbor name: South     Weight: 10806

Node name: South
Node neighbors:
Neighbor name: East      Weight: 3582




### From which region to which region it takes the longest time to travel?

In [13]:
res = []
for frm_area in graph.nodes:
    for to_area in graph.nodes:
        if frm_area == to_area:
            continue
        shortest_path = graph.find_shortest_path(frm_area, to_area)
        if shortest_path:
            res.append((frm_area, to_area, shortest_path, graph.get_path_weight(shortest_path)))
print sorted(res, key=lambda x: x[3])[-1]

('West', 'South', ['West', 'Center', 'East', 'South'], 16063)


## Part III – Non-directional graph Task 1 – define the class
### Implement the NonDirectionalGraph class as a sub-class of Graph.

In [14]:
class NonDirectionalGraph(Graph):
    def add_node(self, node):
        # If node not in graph
        if node not in self:
            # Add node with all its data
            self.nodes[node.name] = node
            # Run through node's neighbors
            for neighbor in node.neighbors:
                # If neighbor not in graph add it
                if not self[neighbor.name]:
                    self.add_node(Node(neighbor.name))
                self[neighbor.name].add_neighbor(node, self.get_edge_weight(node, neighbor))
        else:
            # (node exists)
            # Run through node's neighbors
            for neighbor, weight in node.neighbors.items():
                # If neighbor not in graph add it
                if not self[neighbor.name]:
                    self.add_node(Node(neighbor.name))
                # update weight
                self[node.name].add_neighbor(neighbor, weight)
                self[neighbor.name].add_neighbor(node, weight)

    def add_edge(self, frm_name, to_name, weight=1):
        if frm_name not in self:
            self.add_node(Node(frm_name))
        if to_name not in self:
            self.add_node(Node(to_name))
        self[frm_name].add_neighbor(to_name, weight)
        self[to_name].add_neighbor(frm_name, weight)

    def remove_edge(self, frm_name, to_name):
        if frm_name in self and self[frm_name].is_neighbor(to_name):
            self[frm_name].remove_neighbor(to_name)
            self[to_name].remove_neighbor(frm_name)

## What was the highest number of simultaneous friendships?

In [15]:
def process_line(line, graph):
    words = line.rstrip().split(' ')
    if len(words) == 5 and words[3] == "became" and words[4] == "friends.":
        graph.add_edge(words[0], words[2])
    elif len(words) == 6 and words[3] == "cancelled" and words[4] == "their" and words[5] == "friendship.":
        graph.remove_edge(words[0], words[2])

graph = NonDirectionalGraph("Graph")
res = []
with open("social.txt") as f:
    for line in f:
        process_line(line, graph)

        num_of_friendships = 0
        for node in graph.nodes:
            num_of_friendships += len(graph[node])
        res.append(num_of_friendships/2)

print max(res)

54


## What was the maximum number of friends Reuben had simultaneously?

In [16]:
graph = NonDirectionalGraph("Graph")
res = []
max_Reuben_friends = 0
with open("social.txt") as f:
    for line in f:
        process_line(line, graph)
        try:
            if graph["Reuben"]:
                if len(graph["Reuben"]) > max_Reuben_friends:
                    max_Reuben_friends = len(graph["Reuben"])
        except KeyError:
            # Ruben not in graph
            pass

print max_Reuben_friends

10


## At the current graph (considering all the data of the file), what is the maximal path between nodes in the graph?

In [17]:
graph = NonDirectionalGraph("Graph")
with open("social.txt") as f:
    for line in f:
        process_line(line, graph)

res = []
for frm_name in graph.nodes:
    for to_name in graph.nodes:
        if frm_name == to_name:
            continue
        shortest_path = graph.find_shortest_path(frm_name, to_name)
        if shortest_path:
            res.append(graph.get_path_weight(shortest_path))
print sorted(res)[-1]

2


## Implement a function called suggest_friend(graph, node_name) that returns the name of the node with the highest number of common friends with node_name, which is not already one of his friends.

In [18]:
def suggest_friend(graph, node_name):
    if node_name in graph:
        res = []
        friends_of_node_name = set(graph[node_name].neighbors)
        for node in graph.nodes:
            if node not in friends_of_node_name and node != node_name:
                res.append((node, len(friends_of_node_name & set(graph[node].neighbors))))
        return sorted(res, key=lambda x: x[1])[-1][0]

print suggest_friend(graph, "Reuben")

Judah
